In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Histogram Backprojetion (Indexing via color histograms)
    이미지 히스토그램을 통한 배경투사는 이미지에서 특정 객체를 찾거나 image segmentation, camshift 알고리즘에 자주 사용된다. 관심 대상을 포함하는 이미지의 히스토그램을 만들면, 테스트 이미지 위에 이 히스토그램을 배경투사하여 관심대상의 히스토그램과 얼마나 일치하는지 모든 픽셀의 확률을 계산한다. 확률이 높을 수록 흰색으로 표시된다. 여기서 임계값을 적절하게 조절하여 바이너리 이미지로 만든 후, 원본 이미지와 비트 연산을 통해 배경만 추출 혹은 배경을 제외한 나머지 부분을 추출할 수 있다.

    배경투사하는 과정은 다음과 같다.
    1. roi는 이미지에서 찾을 오브젝트를, target는 오브젝트를 찾을 이미지를 가져온다. 둘 모두 HSV 색공간으로 변환시켜준다.
    2. cv2.calcHist 함수를 이용해서 오브젝트 영역 이미지 히스토그램(M)과 오브젝트 영역을 찾을 이미지 히스토그램(I)을 계산한다. 히스토그램 M과 I의 비율을 계산한다. 이 때 I에 1을 더하지 않으면 배열에 nan값이 생길 수도 있다.
    3. target 이미지의 색공간을 h, s, v 채널로 분리한다. hue와 saturation 값에 R을 팔레트로 사용해서 오브텍트 영역에 있을 확률을 픽셀로 표현하여 새로운 이미지를 생성한다. 이때, 오브젝트 영역에 속할 확률이 높은 픽셀일 수록 흰색으로 표현된다.
    4. 타원 모양 커널로 만들어 이미지와 convolution 하여 이미지 배경투사 결과 영역을 고르게 한다. 이후 배열 B의 타입을 uint8로 변환하고 0~255사이의 값으로 정규화 한다. 마지막으로 오브젝트를 찾은 영역은 흰색, 배경은 검은색이 되도록 threshold 함수를 통해 바이너리 이미지로 변환한다.
    5. 1채널 바이너리 이미지를 3채널 이미지로 변환 후, 원본 이미지 target과 비트연산을 진행하여 겹치는 부분만 얻는다.

In [3]:
roi = cv2.imread('red.png')
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
target = cv2.imread('color_circle.jpg')
hsv_target = cv2.cvtColor(target,cv2.COLOR_BGR2HSV)

M = cv2.calcHist([hsv_roi],[0, 1], None, [180, 256], [0, 180, 0, 256] )
I = cv2.calcHist([hsv_target],[0, 1], None, [180, 256], [0, 180, 0, 256] )
R = M/(I+1)

h,s,v = cv2.split(hsv_target)
B = R[h.ravel(), s.ravel()]
B = np.minimum(B, 1)
B = B.reshape(hsv_target.shape[:2])

disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
B = cv2.filter2D(B, -1, disc)
B = np.uint8(B)
cv2.normalize(B, B, 0, 255, cv2.NORM_MINMAX)
ret,thresh = cv2.threshold(B,20,255,cv2.THRESH_BINARY)

thresh = cv2.merge((thresh,thresh,thresh))
res = cv2.bitwise_and(target,thresh)

cv2.imshow('result1', B)
cv2.imshow('thresh', thresh)
cv2.imshow('result2', res)
cv2.waitKey(0)
cv2.destroyAllWindows()

    openCV에서는 히스토그램의 Backprojection을 계산해주는 cv2.calcBackProject 함수를 제공해준다.
    
## cv2.calcBackProject(images, channels, hist, ranges, scale [, dst]) → dst
    히스토그램의 배경투사를 계산해주는 함수이다. calcHist처럼 각 (x, y)에서 입력받은 이미지에서 선택한 채널의 값을 수집하고 이에 해당하는 히스토그램 빈을 찾는다. 대신 함수는 bin 값을 읽고 scale 값을 통해 그것을 scale한 다음 backProject(x, y)로 저장한다. 통계학적 관점에서 함수는 히스토그램에 의해 나타내어진 확률 분포와 관련하여 각 픽셀의 확률값을 계산한다. 예를들어 어떤 이미지에서 밝은 색 물체를 찾고 추적한다고 해보자.
    
    1. 추적하기전에 물체가 카메라의 전체 프레임을 덮도록 보여준다. Hue 히스토그램을 계산한다. 그럼 히스토그램은 물체의 주 색상과 관련된 가장 큰 값을 얻을 수 있다.
    2. 추적할 때 미리 계산 된 히스토그램을 사용하여 각 입력 이미지의 Hue 평면의 back projection을 계산한다. 약한 색상을 제거하기 위해 임계값을 설정한다. 혹은 채도가 충분하지 않아 너무 어둡거나 너무 밝은 픽셀을 제거하는 것도 의미가 있을 수 있다.
    3. 마지막으로 최종 이미지와 연결된 구성요소(예를들어 가장 큰 값들)를 선택한다.
    이는 CamShift 색상 객체 추적기 알고리즘과 거의 유사하다.
    
    1. images : 물체를 찾을 이미지를 인자로 받는다. CV_8U, CV_16U, CV_32F와 같은 데이터 타입을 가지면서 같은 크기를 가져야 한다. 이미지의 채널 수는 상관 없다. 다만 이미지를 직접적으로 입력하고 싶으면 [] 연산자를 통해 이미지를 리스트화 해야 한다.
    2. channels : back projection을 계산하는 데 사용할 채널의 리스트를 입력으로 받는다.
    3. hist : 물체의 히스토그램을 인자로 받는다.
    4. ranges : range 값으로 측정하고자 하는 픽셀값의 범위를 나타낸다.
    5. scale : 결과 생성시 사용되는 scale factor이다. 보통 default로 1값을 입력한다.

In [5]:
default_x, default_y = -1, -1
mode = False

def onMouse(event, x, y, flag, param):
    global default_x, default_y, mode, img1, img2
    
    if event == cv2.EVENT_LBUTTONDOWN:
        mode = True
        default_x, default_y = x, y
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if mode:
            img1 = img2.copy()
            cv2.rectangle(img1, (default_x, default_y), (x, y), (0, 0, 255), 2)
            cv2.imshow("Original", img1)
            
    elif event == cv2.EVENT_LBUTTONUP:
        if mode:
            mode = False
            w = x - default_x
            h = y - default_y
            
            if w > 0 and h > 0:
                img1 = img2.copy()
                cv2.rectangle(img1, (default_x, default_y), (x, y), (0, 255, 0), 2)
                roi = img1[default_y:y, default_x:x]
                backProjection(img2, roi)
                
    return

def backProjection(img, roi):
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    hsvt = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    roihist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
    cv2.normalize(roihist, roihist, 0, 255, cv2.NORM_MINMAX)
    dst = cv2.calcBackProject([hsvt], [0, 1], roihist, [0, 180, 0, 256], 1)
    
    disc = cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))
    cv2.filter2D(dst, -1, disc, dst)
    
    ret, thr = cv2.threshold(dst, 50, 255, 0)
    thr = cv2.merge((thr, thr, thr))
    res = cv2.bitwise_and(img, thr)
    
    cv2.imshow("backproj", res)
    

img1 = cv2.imread('color_circle.jpg')
img2 = img1.copy()

cv2.namedWindow("Original")
cv2.namedWindow("backproj")
cv2.setMouseCallback("Original", onMouse, param = None)
cv2.imshow("backproj", img2)

while True:
    cv2.imshow("Original", img1)
    k = cv2.waitKey(0) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()